In [8]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
# from nltk.corpus import stopwords  
import nltk
nltk.download('stopwords')
# to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Partha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [9]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [10]:
english_stops = set(stopwords.words('english'))

In [11]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


C:\Users\Partha\AppData\Local\Temp\ipykernel_13624\4067306172.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
29763    [perhaps, wildest, outlier, alfred, hitchcock,...
25437    [though, huge, fan, i, three, stooges, purist,...
2746     [father, son, communicate, little, in, fact, s...
23150    [having, searched, movie, high, low, i, actual...
18916    [this, funny, movie, i, idea, whether, transla...
                               ...                        
20510    [i, dreading, taking, nephews, movie, i, think...
4544     [maybe, aussies, totally, different, sense, hu...
46986    [i, really, looking, forward, watching, film, ...
21540    [show, people, absolutely, delightful, silent,...
30943    [as, psychiatrist, specialized, trauma, i, fin...
Name: review, Length: 40000, dtype: object 

27133    [don, waste, time, money, it, quite, bad, adre...
30429    [this, thing, really, awfull, there, charachte...
35147    [epsilon, k, alien, visitor, i, expected, this...
4831     [the, satirical, movie, website, dateline, hol...
3698     [fantastic, movie, one, excite, senses, is, tr...
 

In [13]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [14]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  293 17648 57062 ... 30223 40014  2316]
 [   70   538   232 ...     0     0     0]
 [  234   384  5639 ...     0     0     0]
 ...
 [    1    14   164 ...     0     0     0]
 [   45    21   323 ...    28   904  5884]
 [  108  4212 14022 ...     0     0     0]] 

Encoded X Test
 [[  331   355    10 ...     0     0     0]
 [    8    65    14 ...     0     0     0]
 [ 1097  1045  8281 ...   616   150   953]
 ...
 [    1   288  5360 ...   339  4155   417]
 [    2 11367 10988 ...     0     0     0]
 [    5   889   541 ...     0     0     0]] 

Maximum review length:  130


In [15]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

d:\VIIT\TY\Sem 5\DL\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [16]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [17]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5506 - loss: 0.6759
Epoch 1: accuracy improved from None to 0.63225, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 71ms/step - accuracy: 0.6323 - loss: 0.6320
Epoch 2/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7746 - loss: 0.5301
Epoch 2: accuracy improved from 0.63225 to 0.74942, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 61ms/step - accuracy: 0.7494 - loss: 0.5423
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.7257 - loss: 0.5440
Epoch 3: accuracy did not improve from 0.74942
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.7321 - loss: 0.5397
Epoch 4/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6443 - loss: 0.6223
Epoch 4: accuracy did not improve from 0.74942
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.6888 - loss: 0.5942
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.6111 - loss: 0.6182
Epoch 5: accuracy did not improve from 0.74942
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 76ms/step - accuracy: 0.6753 - loss: 0.5683


In [20]:
import numpy as np

# Get raw predictions
y_prob = model.predict(x_test, batch_size=128)

# Convert to class labels (index of max prob)
y_pred = np.argmax(y_prob, axis=1)

# If y_test is one-hot, convert it
if y_test.ndim > 1 and y_test.shape[1] > 1:
    y_test_classes = np.argmax(y_test, axis=1)
else:
    y_test_classes = y_test

# Compare predictions
true = np.sum(y_pred == y_test_classes)
wrong = len(y_pred) - true

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(wrong))
print('Accuracy: {:.2f}%'.format(true / len(y_pred) * 100))


79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
Correct Prediction: 4998
Wrong Prediction: 5002
Accuracy: 49.98%


In [21]:
loaded_model = load_model('models/LSTM.h5')

In [22]:
review = str(input('Movie Review: '))

In [24]:
# Pre-process input
import re
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  Titanic
Filtered:  ['titanic']


In [25]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[3354    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [26]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
[[0.9014514]]


In [27]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
